In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeRegressor, plot_tree

### **Data Exploration**

In [50]:
# Load the dataset
home_df = pd.read_csv("home_value.csv")
home_df.info()
home_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 32 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            1460 non-null   int64 
 1   LOTAREA       1460 non-null   int64 
 2   BLDGTYPE      1460 non-null   object
 3   HOUSESTYLE    1460 non-null   object
 4   OVERALLCOND   1460 non-null   int64 
 5   YEARBUILT     1460 non-null   int64 
 6   ROOFSTYLE     1460 non-null   object
 7   EXTERCOND     1460 non-null   object
 8   FOUNDATION    1460 non-null   object
 9   BSMTCOND      1423 non-null   object
 10  HEATING       1460 non-null   object
 11  HEATINGQC     1460 non-null   object
 12  CENTRALAIR    1460 non-null   object
 13  ELECTRICAL    1459 non-null   object
 14  FULLBATH      1460 non-null   int64 
 15  HALFBATH      1460 non-null   int64 
 16  BEDROOMABVGR  1460 non-null   int64 
 17  KITCHENABVGR  1460 non-null   int64 
 18  KITCHENQUAL   1460 non-null   object
 19  TOTRMS

,ID,LOTAREA,BLDGTYPE,HOUSESTYLE,OVERALLCOND,YEARBUILT,ROOFSTYLE,EXTERCOND,FOUNDATION,BSMTCOND,...,GARAGETYPE,GARAGEFINISH,GARAGECARS,GARAGECOND,POOLAREA,POOLQC,FENCE,MOSOLD,YRSOLD,SALEPRICE
0,1,8450,1Fam,2Story,5,2003,Gable,TA,PConc,TA,...,Attchd,RFn,2,TA,0,NaN,NaN,2,2008,208500
1,2,9600,1Fam,1Story,8,1976,Gable,TA,CBlock,TA,...,Attchd,RFn,2,TA,0,NaN,NaN,5,2007,181500
2,3,11250,1Fam,2Story,5,2001,Gable,TA,PConc,TA,...,Attchd,RFn,2,TA,0,NaN,NaN,9,2008,223500
3,4,9550,1Fam,2Story,5,1915,Gable,TA,BrkTil,Gd,...,Detchd,Unf,3,TA,0,NaN,NaN,2,2006,140000
4,5,14260,1Fam,2Story,5,2000,Gable,TA,PConc,TA,...,Attchd,RFn,3,TA,0,NaN,NaN,12,2008,250000


In [51]:
# Count the duplicates
home_df.duplicated().sum()

0

In [52]:
# Get the columns with missing values
missing_columns = home_df.columns[home_df.isnull().any()]
# Count the missing values
home_df[missing_columns].isnull().sum()

BSMTCOND          37
ELECTRICAL         1
FIREPLACEQU      690
GARAGETYPE        81
GARAGEFINISH      81
GARAGECOND        81
POOLQC          1453
FENCE           1179
dtype: int64

In [53]:
# Get the index of missing values
missing_index_1 = home_df[home_df["GARAGETYPE"].isnull()].index
missing_index_2 = home_df[home_df["GARAGEFINISH"].isnull()].index
missing_index_3 = home_df[home_df["GARAGECOND"].isnull()].index

if missing_index_1.all() == missing_index_2.all() == missing_index_3.all():
    print("The missing values are in the same rows.")

The missing values are in the same rows.


In [54]:
# Get the index of missing values
missing_index_4 = home_df[home_df['BSMTCOND'].isnull()].index
# Display the first 5 rows with missing values
home_df.loc[missing_index_4[:5]]

,ID,LOTAREA,BLDGTYPE,HOUSESTYLE,OVERALLCOND,YEARBUILT,ROOFSTYLE,EXTERCOND,FOUNDATION,BSMTCOND,...,GARAGETYPE,GARAGEFINISH,GARAGECARS,GARAGECOND,POOLAREA,POOLQC,FENCE,MOSOLD,YRSOLD,SALEPRICE
17,18,10791,Duplex,1Story,5,1967,Gable,TA,Slab,NaN,...,CarPort,Unf,2,TA,0,NaN,NaN,10,2006,90000
39,40,6040,Duplex,1Story,5,1955,Gable,TA,PConc,NaN,...,NaN,NaN,0,NaN,0,NaN,NaN,6,2008,82000
90,91,7200,1Fam,1Story,5,1950,Gable,TA,Slab,NaN,...,Detchd,Unf,2,TA,0,NaN,NaN,7,2006,109900
102,103,7018,Duplex,1Story,5,1979,Gable,Fa,Slab,NaN,...,Attchd,Unf,2,TA,0,NaN,NaN,6,2009,118964
156,157,7200,1Fam,1Story,7,1950,Hip,TA,CBlock,NaN,...,Detchd,Unf,2,TA,0,NaN,NaN,6,2006,109500


In [55]:
# Count the samples in each class
home_df["BSMTCOND"].value_counts()

BSMTCOND
TA    1311
Gd      65
Fa      45
Po       2
Name: count, dtype: int64

### **Data Preprocessing**

In [56]:
# Fill the missing categorical values with their most frequent value
home_df["BSMTCOND"] = home_df["BSMTCOND"].fillna(home_df["BSMTCOND"].mode()[0])
home_df["ELECTRICAL"] = home_df["ELECTRICAL"].fillna(home_df["ELECTRICAL"].mode()[0])
home_df["GARAGETYPE"] = home_df["GARAGETYPE"].fillna(home_df["GARAGETYPE"].mode()[0])
home_df["GARAGEFINISH"] = home_df["GARAGEFINISH"].fillna(home_df["GARAGEFINISH"].mode()[0])
home_df["GARAGECOND"] = home_df["GARAGECOND"].fillna(home_df["GARAGECOND"].mode()[0])
home_df["FIREPLACEQU"] = home_df["FIREPLACEQU"].fillna(home_df["FIREPLACEQU"].mode()[0])

In [57]:
# Drop the columns with missing values
home_df = home_df.dropna(axis=1)

In [58]:
# Recheck the missing values
missing_columns = home_df.columns[home_df.isnull().any()]

if missing_columns.size == 0:
    print("There are no missing values in the dataset.")

There are no missing values in the dataset.


In [59]:
# Select categorical features except the target variable
categorical_columns = home_df.select_dtypes(include=['object']).columns

le = LabelEncoder()
# Transform the categorical columns into numerical columns
# Use loop to maintain the column names
for column in categorical_columns:
    home_df[column] = le.fit_transform(home_df[column])

home_df.head()

,ID,LOTAREA,BLDGTYPE,HOUSESTYLE,OVERALLCOND,YEARBUILT,ROOFSTYLE,EXTERCOND,FOUNDATION,BSMTCOND,...,FIREPLACES,FIREPLACEQU,GARAGETYPE,GARAGEFINISH,GARAGECARS,GARAGECOND,POOLAREA,MOSOLD,YRSOLD,SALEPRICE
0,1,8450,0,5,5,2003,1,4,2,3,...,0,2,1,1,2,4,0,2,2008,208500
1,2,9600,0,2,8,1976,1,4,1,3,...,1,4,1,1,2,4,0,5,2007,181500
2,3,11250,0,5,5,2001,1,4,2,3,...,1,4,1,1,2,4,0,9,2008,223500
3,4,9550,0,5,5,1915,1,4,0,1,...,1,2,5,2,3,4,0,2,2006,140000
4,5,14260,0,5,5,2000,1,4,2,3,...,1,4,1,1,3,4,0,12,2008,250000


In [60]:
# Standardize the numerical features except the target variable since it is binary
columns = home_df.drop(columns='SALEPRICE').columns

std_scaler = StandardScaler()
home_df[columns] = std_scaler.fit_transform(home_df[columns])
home_df.head()

,ID,LOTAREA,BLDGTYPE,HOUSESTYLE,OVERALLCOND,YEARBUILT,ROOFSTYLE,EXTERCOND,FOUNDATION,BSMTCOND,...,FIREPLACES,FIREPLACEQU,GARAGETYPE,GARAGEFINISH,GARAGECARS,GARAGECOND,POOLAREA,MOSOLD,YRSOLD,SALEPRICE
0,-1.730865,-0.207142,-0.411691,1.026689,-0.517200,1.050994,-0.491516,0.364207,0.835599,0.281048,...,-0.951226,-0.430030,-0.686776,-0.281887,0.311725,0.186019,-0.068692,-1.599111,0.138777,208500
1,-1.728492,-0.091886,-0.411691,-0.543457,2.179628,0.156734,-0.491516,0.364207,-0.549162,0.281048,...,0.600495,1.792425,-0.686776,-0.281887,0.311725,0.186019,-0.068692,-0.489110,-0.614439,181500
2,-1.726120,0.073480,-0.411691,1.026689,-0.517200,0.984752,-0.491516,0.364207,0.835599,0.281048,...,0.600495,1.792425,-0.686776,-0.281887,0.311725,0.186019,-0.068692,0.990891,0.138777,223500
3,-1.723747,-0.096897,-0.411691,1.026689,-0.517200,-1.863632,-0.491516,0.364207,-1.933923,-2.792584,...,0.600495,-0.430030,1.586905,0.950313,1.650307,0.186019,-0.068692,-1.599111,-1.367655,140000
4,-1.721374,0.375148,-0.411691,1.026689,-0.517200,0.951632,-0.491516,0.364207,0.835599,0.281048,...,0.600495,1.792425,-0.686776,-0.281887,1.650307,0.186019,-0.068692,2.100892,0.138777,250000


### **Model Training**

In [61]:
# Split the dataset into features and target variable
X = home_df.drop(columns=['SALEPRICE'])
y = home_df['SALEPRICE']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'Training: {X_train.shape[0]}')
print(f'Testing: {X_test.shape[0]}')

Training: 1168
Testing: 292


In [62]:
# Peform hyperparameter tuning using GridSearchCV
param_grid = {
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
    'splitter': ['best', 'random'],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

model = DecisionTreeRegressor(random_state=42)
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1)
grid_search.fit(X_train, y_train)

c:\Users\Jhondale\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(estimator=DecisionTreeRegressor(random_state=42), n_jobs=-1,
             param_grid={'criterion': ['squared_error', 'friedman_mse',
                                       'absolute_error', 'poisson'],
                         'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'splitter': ['best', 'random']})

In [64]:
# Display the best score and parameters
best_score = grid_search.best_score_
print(f'Best Score: {best_score:.4f}')
grid_search.best_params_

Best Score: 0.6841


{'criterion': 'absolute_error',
 'max_depth': None,
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'splitter': 'random'}

In [66]:
# Get the best model from the grid search
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Calculate the metrics for regression model
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = sqrt(mse)

print("MSE:", mse)
print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

MSE: 1926693530.9023972
MAE: 28961.821917808218
RMSE: 43894.11726988478
R2: 0.7488120480329665


In [71]:
# Get the importance of each feature
importance = best_model.feature_importances_
columns = X.columns

# Create a DataFrame to map the feature names to their importance
importance_df = pd.DataFrame({'Features' : columns, 'Importance' : importance})
importance_df = importance_df.sort_values(by='Importance', ascending=False)
importance_df

,Features,Importance
5,YEARBUILT,0.288189
24,GARAGECARS,0.132497
18,KITCHENQUAL,0.127627
20,FIREPLACES,0.077091
14,FULLBATH,0.060113
1,LOTAREA,0.040685
19,TOTRMSABVGRD,0.038478
22,GARAGETYPE,0.037146
15,HALFBATH,0.027206
3,HOUSESTYLE,0.021332
